In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from torch.utils.tensorboard import SummaryWriter
import os
import json

In [2]:
# Set device
device = torch.device('cuda:1')
# Directory for data and logs
inputdir = '../data/'
logdir = './logs/'
model_dir = './model/'
# TensorBoard writer
writer = SummaryWriter(log_dir=logdir)
# Ensure directory for model checkpoint exists
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

In [11]:
def convert_time_data(data):
    ref_time = pd.Timestamp('2022-01-01 00:00:00')  # Adjust the reference time as needed
    data['t_1h'] = (data['t_1h'] - ref_time).dt.total_seconds() / 3600.0

def load_and_preprocess_data(inputdir):
    # load data
    train_data = pd.read_csv(inputdir + 'loop_sensor_train.csv')
    test_data = pd.read_csv(inputdir + 'loop_sensor_test_x.csv')
    geo_data = pd.read_csv(inputdir + 'geo_reference.csv', delimiter=';')

    # merge geo data
    def parse_coordinates(geo_point):
        latitude, longitude = map(float, geo_point.split(','))
        return pd.Series({'latitude': latitude, 'longitude': longitude})

    geo_data[['latitude', 'longitude']] = geo_data['geo_point_2d'].apply(parse_coordinates)
    train_data = train_data.merge(geo_data, on='iu_ac')
    test_data = test_data.merge(geo_data, on='iu_ac')

    # t_1h convert to seconds
    convert_time_data(train_data)
    convert_time_data(test_data)
    
    scaler = StandardScaler()
    feature_columns = ['t_1h', 'etat_barre']
    train_data[feature_columns] = scaler.fit_transform(train_data[feature_columns])
    test_data[feature_columns] = scaler.transform(test_data[feature_columns])

    return train_data, test_data, geo_data

def create_adjacency_matrix(data):
    # Extract unique nodes and map them to an index
    node_ids = pd.concat([data['iu_ac'], data['iu_nd_amont'], data['iu_nd_aval']]).unique()
    node_index = {node_id: idx for idx, node_id in enumerate(node_ids)}
    # Initialize an adjacency matrix of size NxN where N is the number of unique nodes
    num_nodes = len(node_ids)
    adjacency_matrix = torch.zeros(num_nodes, num_nodes, dtype=torch.float32)
    # Set edges based on upstream and downstream relationships
    for _, row in data.iterrows():
        node_idx = node_index[row['iu_ac']]
        if row['iu_nd_amont'] in node_index:  # Check if upstream node is present
            upstream_idx = node_index[row['iu_nd_amont']]
            adjacency_matrix[upstream_idx][node_idx] = 1  # From upstream to current
        if row['iu_nd_aval'] in node_index:  # Check if downstream node is present
            downstream_idx = node_index[row['iu_nd_aval']]
            adjacency_matrix[node_idx][downstream_idx] = 1  # From current to downstream
    return adjacency_matrix


class TrafficFlowDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.float32)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]


In [13]:
train_data, test_data = load_and_preprocess_data(inputdir)
# Example: Compute adjacency matrix for train data
adjacency_matrix = create_adjacency_matrix(train_data)
# Normalize and prepare dataset
scaler = StandardScaler()
train_features = scaler.fit_transform(train_data[['t_1h', 'etat_barre']])
train_labels = train_data['q']
# Create PyTorch dataset and data loader
train_dataset = TrafficFlowDataset(train_features, train_labels)
train_loader = DataLoader(train_dataset, batch_size=2048, shuffle=True)

ValueError: could not convert string to float: '2022-01-12 12:00:00'

In [4]:
class GraphConvolution(nn.Module):
    def __init__(self, in_features, out_features):
        super(GraphConvolution, self).__init__()
        self.fc = nn.Linear(in_features, out_features)

    def forward(self, x, adj):
        x = torch.matmul(adj, x)  # Apply adjacency matrix
        return torch.relu(self.fc(x))

class TemporalAttentionLayer(nn.Module):
    def __init__(self, num_nodes, num_features, num_timesteps):
        super(TemporalAttentionLayer, self).__init__()
        self.W = nn.Parameter(torch.randn(num_features, num_features))
        self.b = nn.Parameter(torch.randn(num_features))
        self.u = nn.Parameter(torch.randn(num_features))
        self.num_timesteps = num_timesteps

    def forward(self, x):
        # Apply temporal attention across all nodes
        u = torch.tanh(torch.matmul(x, self.W) + self.b)
        att = torch.matmul(u, self.u)
        exps = torch.exp(att)
        alphas = exps / torch.sum(exps, dim=2, keepdim=True)  # softmax over timesteps
        x_tilde = torch.sum(x * alphas.unsqueeze(-1), dim=2)
        return x_tilde

class ASTGCN(nn.Module):
    def __init__(self, num_features, num_nodes, num_timesteps):
        super(ASTGCN, self).__init__()
        self.spatial_gconv = GraphConvolution(num_features, 32)
        self.temporal_attention = TemporalAttentionLayer(num_nodes, 32, num_timesteps)
        self.fc = nn.Linear(32, 1)

    def forward(self, x, adj):
        x = self.spatial_gconv(x, adj)
        x = self.temporal_attention(x)
        x = self.fc(x)
        return x.squeeze()

def save_checkpoint(state, filename='checkpoint.pth.tar'):
    torch.save(state, filename)

def load_checkpoint(model, optimizer, filename='checkpoint.pth.tar'):
    if os.path.isfile(filename):
        checkpoint = torch.load(filename)
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        epoch = checkpoint['epoch']
        print(f"Loaded checkpoint '{filename}' (epoch {checkpoint['epoch']})")
        return epoch
    else:
        print("No checkpoint found at '{}'".format(filename))
        return 0


In [5]:
def train(model, train_loader, criterion, optimizer, device, epochs, model_dir):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for features, labels, adj in train_loader:
            features, labels = features.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(features, adj)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * features.size(0)
        print(f'Epoch {epoch+1}/{epochs}, Loss: {total_loss / len(train_loader.dataset)}')
        torch.save(model.state_dict(), os.path.join(model_dir, 'model_epoch_{}.pth'.format(epoch + 1)))

def evaluate(model, test_loader, device):
    model.eval()
    predictions = []
    with torch.no_grad():
        for features, adj in test_loader:
            features = features.to(device)
            outputs = model(features, adj)
            predictions.extend(outputs.cpu().numpy())
    return predictions


In [ ]:
# Define model, loss function, and optimizer
model = ASTGCN(num_features=2, num_nodes=geo_data.shape[0], num_timesteps=24).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.MSELoss()
# Move model to the correct device
model.to(device)
# Call the training function
train_model(model, train_loader, criterion, optimizer, num_epochs=5)

In [ ]:
# Prepare test data the same way as training data
test_features = scaler.transform(test_data[['t_1h', 'etat_barre']])
test_dataset = TrafficFlowDataset(test_features, np.zeros(len(test_features)))
test_loader = DataLoader(test_dataset, batch_size=2048, shuffle=False)

# Generate predictions
predictions = test_model(model, test_loader)

# Prepare submission file
test_data['estimate_q'] = predictions
submission = test_data[['id', 'estimate_q']]
submission.to_csv(inputdir + 'submission.csv', index=False)
print("Predictions saved to submission.csv")
